# Wald-Wolfowitz Runs Test

Since this repo is concerned with streaks, or runs, it makes sense to conduct a formal runs test to check, whether the "up" and "down" streaks appear to follow the same distribution. We first go through a full implementation with the asymptotic runs test statistic, and then verify our result against the single-sample runs test implementation from the statsmodels library.


In [1]:
import pandas as pd
import math
from scipy.stats import norm
import statsmodels.sandbox.stats.runs as sms
import numpy as np

In [2]:
df1 = pd.read_csv('streaks.csv', header=0)
df1['up'] = df1.up.astype(bool)
df1 = df1[df1['length'] != 0]

In [3]:
df1

,startdate,enddate,length,ret,fwd_ret,up,ticker
0,1990-04-06,1990-05-04,5,-0.400000,0.100000,False,PKOH
1,1990-05-11,1990-05-11,1,0.133333,0.058824,True,PKOH
2,1990-05-18,1990-05-18,1,-0.088235,0.225806,False,PKOH
3,1990-05-25,1990-06-15,4,0.225806,-0.131579,True,PKOH
4,1990-06-22,1990-07-13,4,-0.131579,-0.030303,False,PKOH
...,...,...,...,...,...,...,...
1408148,2019-12-13,2019-12-13,1,0.047252,0.008287,True,GFN
1408149,2019-12-20,2019-12-20,1,-0.026703,0.020814,False,GFN
1408150,2019-12-27,2019-12-27,1,0.037843,-0.005469,True,GFN
1408151,2020-01-03,2020-01-17,3,-0.016408,NaN,False,GFN


With the table set up, we first calculate the total number of runs, `R`.

In [4]:
R = df1.startdate.count()
R

1408078

Next, we need the total number of upward (downward) weeks, `n_u` (`n_d`), which is the sum product of the `up` (`down`) and `length` columns in dataframe `tbl`.

In [5]:
n_d = df1[df1['up'] == False].length.sum()
n_u = df1[df1['up'] == True].length.sum()
print(f'n_u = {int(n_u):d}, n_d = {int(n_d):d}')

n_u = 1391916, n_d = 1359737


In [6]:
n_u = float(n_u)  # This is done to avoid address overflow errors with integer types, since terms in the 
n_d = float(n_d)  # following calculations get very large

The formula for the expected value and variance of `R` can be found, for example, [here](https://online.stat.psu.edu/stat414/node/329/). I enter it in the following:

In [7]:
R_hat = 2 * n_u * n_d / (n_u + n_d) + 1
R_hat

1375639.3425468255

In [8]:
R_var = 2 * n_u * n_d * (2 * n_u * n_d - n_u - n_d) / ((n_u + n_d)**2 * (n_u + n_d - 1))

Next, we calculate the asymptotic standard normal test statistic, `Z`:

In [9]:
Z = (R - R_hat)/math.sqrt(R_var)
Z

39.116096720358094

In [10]:
norm.cdf(Z)

1.0

We now use the normal CDF at `Z` to obtain the confidence bound `1 - alpha`

In [11]:
alpha = 2 * (norm.cdf(Z) -1)

In [12]:
alpha

0.0

The test statistic leads to the conclusion that the up and down streaks follow different distributions.

## 2nd Implementation

In order to confirm this result, we recalculate using the single-sample runs test implemented in the statsmodel standard library. This implementation requires a single array of zeroes and ones as input. So we start over, by replicating each row in the data exactly `length` times:

In [13]:
newdf = pd.DataFrame(np.repeat(df1.values,df1.length,axis=0))
newdf.columns = df1.columns  # recreate column names
newdf.head(20)

,startdate,enddate,length,ret,fwd_ret,up,ticker
0,1990-04-06,1990-05-04,5,-0.4,0.1,False,PKOH
1,1990-04-06,1990-05-04,5,-0.4,0.1,False,PKOH
2,1990-04-06,1990-05-04,5,-0.4,0.1,False,PKOH
3,1990-04-06,1990-05-04,5,-0.4,0.1,False,PKOH
4,1990-04-06,1990-05-04,5,-0.4,0.1,False,PKOH
5,1990-05-11,1990-05-11,1,0.133333,0.0588235,True,PKOH
6,1990-05-18,1990-05-18,1,-0.0882353,0.225806,False,PKOH
7,1990-05-25,1990-06-15,4,0.225806,-0.131579,True,PKOH
8,1990-05-25,1990-06-15,4,0.225806,-0.131579,True,PKOH
9,1990-05-25,1990-06-15,4,0.225806,-0.131579,True,PKOH


In [14]:
x = newdf.up.astype(int) # turn the boolean 'up' column into zeros and ones
x.head(10)

0    0
1    0
2    0
3    0
4    0
5    1
6    0
7    1
8    1
9    1
Name: up, dtype: int64

In [15]:
sms.runstest_1samp(x, correction=False)

(36.48975933941316, 1.6119120821411254e-291)

While the Z statistic is somewhat different, likely due to use of the "exact" instead of "asymptotic" version in statsmodel, the end result remains the same. **The null hypothesis that the streaks are randomly distributed has to be discarded.**